In [1]:
pip install pandas networkx[default] -U matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 5.1 MB/s 
     |████████████████████████████████| 9.4 MB 43.2 MB/s 
     |████████████████████████████████| 965 kB 44.8 MB/s 
     |████████████████████████████████| 295 kB 47.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
# Load data

url = "https://media.githubusercontent.com/media/adeindriawan/nft-fraud-detection/master/transfers.csv"
df = pd.read_csv(url)
df['label'] = 'nonfraud'
print(df)

                                     event_id  \
0        cd816651-56b2-4ed9-887c-c83de732428d   
1        82cc5228-eb80-4e0d-9f6f-e644dec3ab06   
2        6e1f9cc4-d1df-4a6b-972d-a20765beb326   
3        43dc27a7-a72d-4894-809c-e868de05f7ee   
4        47b7839b-9b87-442d-b2c1-9ebedcad8e06   
...                                       ...   
4514724  d45af108-4869-44bb-833d-2539faba0f1b   
4514725  73c81c59-c17a-4154-8bbc-5d72a27ee905   
4514726  27f9c904-06cf-4807-9011-d5ecae274249   
4514727  89718578-d85b-493b-ab2f-8bd49187109c   
4514728  a8f8314a-36c5-4999-88e6-86dc4b3e347a   

                                          transaction_hash  block_number  \
0        0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...      12936373   
1        0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...      12936373   
2        0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...      12936373   
3        0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...      12936373   
4        0x0c73daad5e6946e6b2f1

In [4]:
threshold = 5
multiple_trx = df[['nft_address', 'token_id']].groupby(['nft_address', 'token_id']).size().loc[lambda x: x > threshold].reset_index(name='freq').sort_values(by=['freq'], ascending=False)
print(multiple_trx)

                                      nft_address  \
15587  0xACd3CF818EFe8ddce84C585ddCB147c4C844D3b3   
22702  0xb7F7F6C52F2e2fdb1963Eab30438024864c313F6   
7709   0x60F80121C31A0d46B5279700f9DF786054aa5eE5   
22623  0xb7F7F6C52F2e2fdb1963Eab30438024864c313F6   
22768  0xb7F7F6C52F2e2fdb1963Eab30438024864c313F6   
...                                           ...   
10210  0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205   
16993  0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D   
16994  0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D   
10209  0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205   
16584  0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D   

                                                token_id  freq  
15587                                                  0   272  
22702                                                639    72  
7709                                             1208910    72  
22623                                               3920    67  
22768                                 

In [ ]:
from networkx import simple_cycles

for i, v in enumerate(multiple_trx.iterrows()):
  filter1 = df['nft_address'] == v[1]['nft_address']
  filter2 = df['token_id'] == v[1]['token_id']
  f_m_t = df[filter1 & filter2]
  f_m_t_sorted = f_m_t.sort_values(by=['timestamp'], ascending=True)
  
  G = nx.DiGraph()
  # unique_addresses = (f_m_t_sorted['from_address'].append(f_m_t_sorted['to_address'])).unique()
  unique_addresses = pd.concat([f_m_t_sorted['from_address'], f_m_t_sorted['to_address']]).unique()
  account_nodes = [tuple((x, {"entity": "Account", "address": x})) for x in unique_addresses]
  G.add_nodes_from(account_nodes)
  for index, item in f_m_t.iterrows():
    G.add_edge(item['from_address'], item['to_address'], rel='TRANSFERED_TO')

  loops = list(simple_cycles(G))
  if len(loops) >= 2:
    df.loc[filter1 & filter2, 'label'] = 'fraud'

  print(i)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [ ]:
fraud_label_filter = df['label'] == 'fraud'
df[fraud_label_filter]

CONTOH DATA LABELING

In [ ]:
f1 = df['nft_address'] == '0x053afa4D216F20DD0c15631Fb1Fb26a02B152c09'
f2 = df['token_id'] == '1'

df.loc[f1 & f2, 'label'] = 'fraud'
df[f1 & f2]

In [5]:
ex = '0x053afa4D216F20DD0c15631Fb1Fb26a02B152c09_1.csv'
ex_df = pd.read_csv(ex)
ex_df_sorted = ex_df.sort_values(by=['timestamp'], ascending=True)
print(ex_df_sorted)

                                event_id  \
0   1ec14616-f732-4a10-9053-9dd7009cf071   
23  abbbb140-8780-4c81-8650-b2d97a74d2d7   
21  063a27e4-2ca5-4d05-b3d5-27378ae826b8   
20  b5a8a3da-e35b-4117-bd4f-d3ac0716e82d   
19  7eb5030b-ae71-4d7f-9f03-a2b5d4daa7c2   
18  b6c6a0cd-a095-4491-8e4b-08c1e61d3b66   
17  531db15b-2229-46f4-b1bd-a1d921ba6c83   
16  911f528f-9320-413c-a2aa-73e7371b1b25   
15  c36dc2f7-eca7-4476-bb25-987aeb0e4273   
14  c3270793-eb91-4260-b9c0-ab37d005ed6e   
13  bb3d6461-c078-461b-9108-fbb787483a30   
12  595d94c7-1a00-487a-acd2-378fc827b2e4   
22  0d050eaf-f10f-4383-bcb1-5fb0ee6b77e2   
10  d6341027-c026-456f-a9f6-93d4eb867e4b   
9   08f99d63-522d-4593-9760-318e42f583dc   
8   72cc5ee5-7ef2-4581-b24f-832d5ae7e671   
7   b43f2046-6dfa-41d2-ac94-5f3ca958af8c   
6   59b7c882-ddb0-42f2-a81e-1235e253cb88   
5   9e8792ae-a581-4099-a513-4dd080d21c4b   
4   142b1435-526a-4cb0-b328-548199df8da5   
3   21b93550-0922-447b-8d82-ff8e3d124bda   
2   b235da12-fc5f-4250-91af-3118

In [6]:
G = nx.DiGraph()

In [14]:
unique_addresses = pd.concat([ex_df_sorted['from_address'], ex_df_sorted['to_address']]).unique()
account_nodes = [tuple((x, {"entity": "Account", "address": x})) for x in unique_addresses]
G.add_nodes_from(account_nodes)
print(account_nodes)

[('0x5b411285bDE8a6E0237B465B5Cb6577040885c98', {'entity': 'Account', 'address': '0x5b411285bDE8a6E0237B465B5Cb6577040885c98'}), ('0xA731178E11A70d65BDEeD6f6575cD2ccfD9C265A', {'entity': 'Account', 'address': '0xA731178E11A70d65BDEeD6f6575cD2ccfD9C265A'}), ('0x99988890F2DE1d59e1EB76d2f34CDE371044b8C8', {'entity': 'Account', 'address': '0x99988890F2DE1d59e1EB76d2f34CDE371044b8C8'}), ('0x281e8D319088a7BCf99C3F98E6C145e9Be62cF66', {'entity': 'Account', 'address': '0x281e8D319088a7BCf99C3F98E6C145e9Be62cF66'}), ('0xe2ca7390e76c5A992749bB622087310d2e63ca29', {'entity': 'Account', 'address': '0xe2ca7390e76c5A992749bB622087310d2e63ca29'}), ('0xF962f8D31b4f5e9794C76658b8E09aE08B4F0506', {'entity': 'Account', 'address': '0xF962f8D31b4f5e9794C76658b8E09aE08B4F0506'}), ('0xea5ABc1a1689984EbFDC41130886BdAEB5c24078', {'entity': 'Account', 'address': '0xea5ABc1a1689984EbFDC41130886BdAEB5c24078'}), ('0x9428E55418755b2F902D3B1f898A871AB5634182', {'entity': 'Account', 'address': '0x9428E55418755b2F902D

In [ ]:
node_pos=nx.spring_layout(G) #Any layout will work here, including nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos=node_pos, node_color='r')#draw nodes
plt.show() 

In [16]:
for index, item in ex_df_sorted.iterrows():
  G.add_edge(item['from_address'], item['to_address'], rel='TRANSFERED_TO')

In [ ]:
nodes = G.nodes()

pos = nx.spring_layout(G)
ec = nx.draw_networkx_edges(G, pos, edge_color='r')
nc = nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color='b', 
                            node_size=100, cmap=plt.cm.jet)

plt.colorbar(nc)
plt.axis('off')
plt.show()

In [18]:
from networkx import simple_cycles
loops = list(simple_cycles(G))
print(len(loops))

9
